In [1]:
import numpy as np
import pandas as pd
import json, requests

In [2]:
persons = {}
courses = {}
subjects = {}
entries = {}
college_list = {
    'Ambedkar Institute Of Advanced Communication Technologies & Research (Formerly Ait)' : 'AIACTR', 
    'Amity School Of Engineering & Technology' : 'ASET',
    'Bhagwan Mahaveer College Of Engineering & Management' : 'BMCEM',
    'Bhagwan Parshuram Institute Of Technology' : 'BPIT',
    'Bharati Vidyapeeth College Of Engineering' : 'BVCOE',
    'Bm Institute Of Engineering & Technology' : 'BMIET',
    'Delhi Technical Campus, Greater Noida' : 'DTC',
    'Dr. Akhilesh Das Gupta Institute Of Technology And Management (Formerly Niec)' : 'ADGITM',
    'Dr. Akhilesh Das Gupta Institute Of Technology And Management(Formerly Niec)' : 'ADGITM',
    'G B Pant Govt. Engineering College' : 'GBPGEC',
    'Guru Tegh Bahadur Institute Of Technology' : 'GTBIT',
    'Hmr Institute Of Technology & Management' : 'HMRITM',
    'Jims Engineering Management Technical Campus, Greater Noida' : 'JIMS',
    'Maharaja Agrasen Institute Of Technology' : 'MAIT',
    'Maharaja Surajmal Institute Of Technology' : 'MSIT',
    'Mahavir Swami Institute Of Technology' : 'MASW',
    'Northern India Engineering College' : 'ADGITM'
}

i = 1
with open('outputf.in') as f:
    c_institute = ''
    c_course_code = '' 
    c_course_name = '' 
    c_enrollment_no = ''
    c_sem = ''
    c_type = ''
    c_batch = ''
    while 1:
        inp = f.readline().split()
        if len(inp) < 1: 
            continue
        if inp[0] == '1':
            c_institute = college_list[' '.join(inp[2:]).title()]
        elif inp[0] == '2':
            inp[1] = int(inp[1])
            if inp[1] not in subjects:
                if inp[-1] == '/':
                    inp[-1] = '1'
                subjects[inp[1]] = {
                    'n' : ' '.join(inp[3:-1]).title(), # subject_name
                    'v' : int(inp[-1]),
                    's' : int(c_sem)
                }
        elif inp[0] == '3':
            c_enrollment_no = inp[1]
            if inp[1] not in persons:
                persons[inp[1]] = {
                    'n' : ' '.join(inp[2:]), # name
                    'c' : c_institute, # college
                    'e' : int(c_enrollment_no), # enrollment_no
                    's' : int(c_enrollment_no[3:6]), # class_no
                    'b' : int(c_batch), # batch
                    'z' : int(c_course_code) # course_code
                }
        elif inp[0] == '4':
            for x in range(len(inp)):
                if inp[x][0] in ['A','B','C','D','-']:
                    inp[x] = '0'
            entries.setdefault(c_enrollment_no, []).append({
                'm' : int(c_sem), # semester
                'u' : int(inp[1][:-3]), # subject_code
                'n' : int(inp[2]), # internal
                'x' : int(inp[3]), # external
                't' : int(inp[4].split('(')[0]), # total
                'f' : (c_type == 'REGULAR') # type
            })
        elif inp[0] == '5':
            c_sem = int(inp[2])
        elif inp[0] == '6':
            c_type = inp[2]
        elif inp[0] == '7':
            c_course_code = int(inp[2])
        elif inp[0] == '8':
            c_course_name = ' '.join(inp[1:])
            courses[c_course_code] = c_course_name
        elif inp[0] == '9':
            c_batch = int(inp[2])
        i+=1
        if i==138074:
            break

In [3]:
with open('persons.json', 'w') as f:
    json.dump(persons, f)
with open('subjects.json', 'w') as f:
    json.dump(subjects, f)
with open('courses.json', 'w') as f:
    json.dump(courses, f)
with open('entries.json', 'w') as f:
    json.dump(entries, f)

In [4]:
with open('persons.json', 'r') as f:
    persons = json.load(f)
with open('subjects.json', 'r') as f:
    subjects = json.load(f)
with open('courses.json', 'r') as f:
    courses = json.load(f)
with open('entries.json', 'r') as f:
    entries = json.load(f)

In [5]:
studentwise = []
for x in persons:
    student = {}
    student['en_no'] = x
    student['name'] = persons[x]['n'].title()
    student['college'] = persons[x]['c']
    student['batch'] = persons[x]['b']
    student['class'] = persons[x]['s']
    student['course'] = persons[x]['z']
    tot,totw,ns,nsw = [0]*7,[0]*7,[0]*7,[0]*7
    num_sems = set()

    for y in [1,2,3,4,5,6]:
        for z in entries[x]:
            if z['m'] == y:
                num_sems.add(y)
                student[z['u']] = (z['n'], z['x'], z['t'])
                tot[y] += z['t']
                ns[y] += 1
                totw[y] += z['t'] * subjects[str(z['u'])]['v']
                nsw[y] += subjects[str(z['u'])]['v']
                
    student['sems'] = list(num_sems)
    for y in num_sems:
        student['tot'+str(y)] = tot[y]
        student['totw'+str(y)] = totw[y]
        student['ns'+str(y)] = ns[y]
        student['nsw'+str(y)] = nsw[y]
    studentwise.append(student)

In [6]:
with open('all_data.json', 'w') as f:
    json.dump(studentwise, f)

In [7]:
df = pd.DataFrame(studentwise).set_index('en_no')